# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: October 7th 2025

**Student Name**: Andre Jair Sanchez Contreras

**Professor**: Pablo Camarillo Ramirez

In [ ]:
from AndreModule.random_logs import random_logs
from pcamarillor.spark_utils import SparkUtils

In [ ]:
#The pipeline should monitor a directory for simulated server log files, analyze error patterns in real time, and filter alerts for critical issues
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab07") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [ ]:
!mkdir -p /opt/spark/work-dir/data/logs_stream/

In [ ]:
!ls /opt/spark/work-dir/data/logs_stream/

In [ ]:
#Display at least three micro-batches of the streaming process.
from pyspark.sql.functions import split, element_at, trim, col

SRC = "/opt/spark/work-dir/data/logs_stream/"
names_schema = SparkUtils.generate_schema([("Level", "string"), ("Message", "string")])

logs_df = (spark.readStream
    .schema(logs_schema)
    .format("json")
    .load(SRC))

#filtrado
filtered = (logs_df
    .filter(logs_df.level.isin("INFO", "WARN", "ERROR"))
    .select("level", "message"))


query = filtered.writeStream \
            .outputMode("append") \
            .format("console") \
            .start()

#3 micro-batches
query.awaitTermination(30)
